-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Evaluating Large Language Models (LLMs)
This notebook demonstrates methods for evaluating LLMs.  We focus on the task of summarization and cover accuracy, ROUGE-N, and perplexity.

### ![Dolly](https://files.training.databricks.com/images/llm/dolly_small.png) Learning Objectives
1. Know how to compute ROUGE-N and other metrics.
2. Gain an intuitive understanding of ROUGE-N.
3. Test various models and model sizes on the same data, and compare their results.

## Classroom Setup

In [0]:
%pip install rouge_score==0.1.2

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached rouge_score-0.1.2-py3-none-any.whl
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
%run ../Includes/Classroom-Setup

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


Resetting the learning environment:
| enumerating serving endpoints...found 0...(0 seconds)
| removing the working directory "dbfs:/mnt/dbacademy-users/vijaymohire@bhadaleit.onmicrosoft.com/large-language-models"...(2 seconds)

Skipping install of existing datasets to "dbfs:/mnt/dbacademy-datasets/large-language-models/v01"


Importing lab testing framework.



Using the "default" schema.

Predefined paths variables:
| DA.paths.working_dir: /dbfs/mnt/dbacademy-users/vijaymohire@bhadaleit.onmicrosoft.com/large-language-models
| DA.paths.user_db:     /dbfs/mnt/dbacademy-users/vijaymohire@bhadaleit.onmicrosoft.com/large-language-models/database.db
| DA.paths.datasets:    /dbfs/mnt/dbacademy-datasets/large-language-models/v01

Setup completed (6 seconds)

The models developed or used in this course are for demonstration and learning purposes only.
Models may occasionally output offensive, inaccurate, biased information, or harmful instructions.


## How can we evaluate summarization?

Suppose you are developing a smartphone news app and need to display automatically generated summaries of breaking news articles.  How can you evaluate whether or not the summaries you are generating are good?

![](https://drive.google.com/uc?export=view&id=1V6cMD1LgivCb850JDhva1DO9EWVH8rJ7)

## Dataset

We will use a subset of the `cnn_dailymail` dataset from See et al., 2017, downloadable from the [Hugging Face `datasets` hub](https://huggingface.co/datasets/cnn_dailymail).

This dataset provides news article paired with summaries (in the "highlights" column).  Let's load the data and take a look at some examples.

In [0]:
import torch
from datasets import load_dataset

full_dataset = load_dataset(
    "cnn_dailymail", version="3.0.0", cache_dir="/dbfs/mnt/dbacademy-datasets/large-language-models/v01"
)  # Note: We specify cache_dir to use pre-cached data.

# Use a small sample of the data during this lab, for speed.
sample_size = 100
sample = (
    full_dataset["train"]
    .filter(lambda r: "CNN" in r["article"][:25])
    .shuffle(seed=42)
    .select(range(sample_size))
)
sample

/databricks/python_shell/dbruntime/huggingface_patches/datasets.py:13: UserWarning: During large dataset downloads, there could be multiple progress bar widgets that can cause performance issues for your notebook or browser. To avoid these issues, use `datasets.utils.logging.disable_progress_bar()` to turn off the progress bars.
  warnings.warn(


Found cached dataset cnn_dailymail (/dbfs/mnt/dbacademy-datasets/large-language-models/v01/cnn_dailymail/default/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


  0%|          | 0/3 [00:00<?, ?it/s]

Filter:   0%|          | 0/287113 [00:00<?, ? examples/s]

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 100
})

In [0]:
display(sample.to_pandas())

article highlights id (CNN) -- A magnitude 6.7 earthquake rattled Papua New Guinea early Friday afternoon, according to the U.S. Geological Survey. The quake was centered about 200 miles north-northeast of Port Moresby and had a depth of 28 miles. No tsunami warning was issued, according to the Tsunami Warning Center. Papua New Guinea is on the so-called Ring of Fire, an arc of fault lines circling the Pacific Basin that is prone to frequent earthquakes and volcanic eruptions. Papua New Guinea is on the so-called Ring of Fire .
It's on an arc of fault lines that is prone to frequent earthquakes .
No tsunami warning was issued . 8093dba7bc2260c26f18939826909ef27549c758 (CNN) -- Pakistan took big steps towards leveling the two-Test cricket series against Australia after bowling out their opponents for just 88 runs in Britain on Wednesday. The Australians, who won the first match by 150 runs, failed to reach three figures in an innings for the first time since 1984 as captain Ricky Ponting paid for his decision to bat first in bowler-friendly conditions after winning the toss. Pakistan, led by new captain Salman Butt after Shahid Afridi ended his brief return to the Test arena following that defeat at Lord's, then reached 148-3 before rain cut short the opening day in Leeds. Teenage paceman Mohammad Aamer and fellow opening bowler Mohammad Asif took three wickets each as Australia crashed to the nation's lowest first-innings score since 1956 -- when dismissed for 80, also by Pakistan. "There was moisture under the wicket so I think it was a shocking decision, especially for Australia," Pakistan bowler Umar Gul told reporters. "I don't know what the captain and coach's decision was going to be but if we won the toss we were ready to bowl first. This morning it was swinging a lot and the ball was seaming." Tim Paine's leading knock of 17 was Australia's fourth-lowest in a Test innings in which all 11 players have batted. The No. 7 was last man out in a fighting 47-ball stay, scoring one more run than all-rounder Marcus North. Gul also claimed two wickets while Umar Amin dismissed North and was also involved in the run-out of tailender Ben Hilfenhaus. The Pakistanis went into a 13-match losing run against Australia that goes back to 1995, but struck an early blow when the 18-year-old Aamer trapped opener Simon Katich leg before wicket for 13. Afridi quits Tests after 150-run defeat at Lord's . That opened the floodgates as Asif sent Shane Watson (5) and Ponting (6) back to the pavilion at Headingley in similar fashion, and Gul bowled Michael Clarke (3). From 29-4, Australia slumped further to 73-6 at lunch as Mike Hussey (5) also went lbw to Gul and North was caught behind off the medium pace of Amin. Aamer then took wickets with the first two balls of the middle session, bowling Steven Smith and Mitchell Johnson, but Hilfenhaus avoided becoming the hat-trick victim. Australia's bowlers could not extract the same amount of swing from the overcast conditions, and Butt put on 80 for the first wicket with Imran Farhat before being bowled by Hilfenhaus. Farhat also fell short of a half-century as he was trapped lbw by Watson for 45 with the score at 133, and the all-rounder then dispatched Azar Ali (30) seven runs later. Amin (1) and Umar Akmal (8) were unbeaten when bad light and then rain ended a day on which 13 wickets fell for just 236 runs. The series is being played in England due to security issues in Pakistan. Australia collapse to 88 all out on opening day of second Test against Pakistan in Leeds .
Pakistan, seeking to level two-match series, reached 148-3 when bad light stopped play .
Australia captain Ricky Ponting surprisingly chose to bat first in overcast conditions .
His team failed to reach three figures in a test for the first time since 1984 . 67d626156f971d0bf55e5f2a48e1ed965eb622a6 (CNN) -- Federal prosecutors are pushing to force the Arizona man accused of fatally gunning down six people and wounding 13 others, incl

In [0]:
example_article = sample["article"][0]
example_summary = sample["highlights"][0]
print(f"Article:\n{example_article}\n")
print(f"Summary:\n{example_summary}")

Article:
(CNN) -- A magnitude 6.7 earthquake rattled Papua New Guinea early Friday afternoon, according to the U.S. Geological Survey. The quake was centered about 200 miles north-northeast of Port Moresby and had a depth of 28 miles. No tsunami warning was issued, according to the Tsunami Warning Center. Papua New Guinea is on the so-called Ring of Fire, an arc of fault lines circling the Pacific Basin that is prone to frequent earthquakes and volcanic eruptions.

Summary:
Papua New Guinea is on the so-called Ring of Fire .
It's on an arc of fault lines that is prone to frequent earthquakes .
No tsunami warning was issued .


## Summarization

In [0]:
import pandas as pd
import torch
import gc
from transformers import AutoTokenizer, T5ForConditionalGeneration

In [0]:
def batch_generator(data: list, batch_size: int):
    """
    Creates batches of size `batch_size` from a list.
    """
    s = 0
    e = s + batch_size
    while s < len(data):
        yield data[s:e]
        s = e
        e = min(s + batch_size, len(data))


def summarize_with_t5(
    model_checkpoint: str, articles: list, batch_size: int = 8
) -> list:
    """
    Compute summaries using a T5 model.
    This is similar to a `pipeline` for a T5 model but does tokenization manually.

    :param model_checkpoint: Name for a model checkpoint in Hugging Face, such as "t5-small" or "t5-base"
    :param articles: List of strings, where each string represents one article.
    :return: List of strings, where each string represents one article's generated summary
    """
    if torch.cuda.is_available():
        device = "cuda:0"
    else:
        device = "cpu"

    model = T5ForConditionalGeneration.from_pretrained(
        model_checkpoint, cache_dir=DA.paths.datasets
    ).to(device)
    tokenizer = AutoTokenizer.from_pretrained(
        model_checkpoint, model_max_length=1024, cache_dir=DA.paths.datasets
    )

    def perform_inference(batch: list) -> list:
        inputs = tokenizer(
            batch, max_length=1024, return_tensors="pt", padding=True, truncation=True
        )

        summary_ids = model.generate(
            inputs.input_ids.to(device),
            attention_mask=inputs.attention_mask.to(device),
            num_beams=2,
            min_length=0,
            max_length=40,
        )
        return tokenizer.batch_decode(summary_ids, skip_special_tokens=True)

    res = []

    summary_articles = list(map(lambda article: "summarize: " + article, articles))
    for batch in batch_generator(summary_articles, batch_size=batch_size):
        res += perform_inference(batch)

        torch.cuda.empty_cache()
        gc.collect()

    # clean up
    del tokenizer
    del model
    torch.cuda.empty_cache()
    gc.collect()
    return res

In [0]:
t5_small_summaries = summarize_with_t5("t5-small", sample["article"])

In [0]:
reference_summaries = sample["highlights"]

In [0]:
display(
    pd.DataFrame.from_dict(
        {
            "generated": t5_small_summaries,
            "reference": reference_summaries,
        }
    )
)

generated,reference
"a magnitude 6.7 earthquake rattles Papua new Guinea early Friday afternoon. no tsunami warning was issued, according to the Tsunami Warning Center. papu",Papua New Guinea is on the so-called Ring of Fire . It's on an arc of fault lines that is prone to frequent earthquakes . No tsunami warning was issued .
the two-Test cricket series is being played in England due to security issues. the series is being played in England due to security issues in Pakistan. the series is being played in,"Australia collapse to 88 all out on opening day of second Test against Pakistan in Leeds . Pakistan, seeking to level two-match series, reached 148-3 when bad light stopped play . Australia captain Ricky Ponting surprisingly chose to bat first in overcast conditions . His team failed to reach three figures in a test for the first time since 1984 ."
federal prosecutors want jared Lee Loughner to submit a handwriting sample. prosecutors want the sample to compare with handwritten notes found in his residence. the federal grand,Jared Loughner is refusing the government's request for a writing sample . Authorities want it to compare with notes found in his home after the shooting . Loughner faces 49 charges related to a mass shooting outside a Tucson market .
"new: ""he tried to kill people,"" a 17-year-old student says. new: ""you could feel in the room all the anger that everyone had for him,"" she","Shooting victim McKayla Hicks went to hearing for accused killer James Holmes . She said she could feel ""all the anger that everyone had for"" Holmes . The incident has changed her, said Hicks . A bullet lodged in Hicks' jaw -- doctors said it is safer to leave it there ."
double-amputee sprinter Oscar Pistorius will compete at the 2012 able bodied Olympics. the four-time paralympic gold medalist won a silver,Oscar Pistorius to become first double-amputee Olympian in London . The 25-year-old has been selected in the individual 400 and 4x400m relay . Pistorius had both of his legs amputated when he was 11 months old . He won a silver medal at last year's World Championships in the 4 x 400m relay .
new: a grand jury indicted the governor on charges of coercion of a public servant and abuse of his capacity. new: a grand jury indicted the,"NEW: Perry lawyer calls indictments ""political abuse of the court system"" Indictment by country grand jury in Texas stems from effort to remove local prosecutor . Perry allegedly threatened to veto funding for a program run by the DA in Austin . Indictment could have political implications ."
prosecutor says there is enough evidence to continue an investigation. prosecutor Alberto Nisman alleged that he covered up the 1994 bombing. he was found dead in,"Prosecutor to judge: Enough evidence for investigation of President to continue . Different prosecutor, before he died, alleged President hid Iran's alleged involvement in bombing . President Cristina Fernandez de Kirchner, other officials deny cover-up ."
"UEFA says it is acting over ""the setting-off and throwing of fireworks"" UEFA president calls fans to ""conduct themselves with dignity and respect"" UEFA has already",NEW: UEFA president Michel Platini urges fans to behave at decisive matches Saturday . UEFA says there were racist chants from Croatian fans during a match against Italy . The issue of racism threatens to mar the Euro 2012 soccer tournament . A disciplinary panel will consider the cause against Croatia on Tuesday .
new: a new high-level group will discuss economic and trade relations between the two nations. new: the presidents will meet with the president on friday. new: the,A new high-level group to discuss economic cooperation will convene in the fall . Obama says ties between the U.S. and Mexico go beyond security and immigration . Mexico's president says his administration is committed to fighting organized crime . The U.S. president will travel to Costa Rica on Friday to meet with Central American leaders .
"police, other

You may see some warning messages in the output above.  While pipelines are handy, they provide less control over the tokenizer and model; we will dive deeper later.

But first, let's see how our summarization pipeline does!  We'll compute 0/1 accuracy, a classic ML evaluation metric.

In [0]:
accuracy = 0.0
for i in range(len(reference_summaries)):
    generated_summary = t5_small_summaries[i]
    if generated_summary == reference_summaries[i]:
        accuracy += 1.0
accuracy = accuracy / len(reference_summaries)

print(f"Achieved accuracy {accuracy}!")

Achieved accuracy 0.0!


Accuracy zero?!?  We can see that the (very generic) metric of 0/1 accuracy is not useful for summarization.  Thinking about this more, small variations in wording may not matter much, and many different summaries may be equally valid.  So how can we evaluate summarization?

## ROUGE

Now that we can generate summaries---and we know 0/1 accuracy is useless here---let's look at how we can compute a meaningful metric designed to evaluate summarization: ROUGE.

Recall-Oriented Understudy for Gisting Evaluation (ROUGE) is a set of evaluation metrics designed for comparing summaries from Lin et al., 2004.  See [Wikipedia](https://en.wikipedia.org/wiki/ROUGE_&#40;metric&#41;) for more info.  Here, we use the Hugging Face Evaluator wrapper to call into the `rouge_score` package.  This package provides 4 scores:

* `rouge1`: ROUGE computed over unigrams (single words or tokens)
* `rouge2`: ROUGE computed over bigrams (pairs of consecutive words or tokens)
* `rougeL`: ROUGE based on the longest common subsequence shared by the summaries being compared
* `rougeLsum`: like `rougeL`, but at "summary level," i.e., ignoring sentence breaks (newlines)

In [0]:
import evaluate
import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt")

rouge_score = evaluate.load("rouge")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


You can call `rouge_score` evaluator directly, but we provide a convenience function below to handle the expected input format.

In [0]:
def compute_rouge_score(generated: list, reference: list) -> dict:
    """
    Compute ROUGE scores on a batch of articles.

    This is a convenience function wrapping Hugging Face `rouge_score`,
    which expects sentences to be separated by newlines.

    :param generated: Summaries (list of strings) produced by the model
    :param reference: Ground-truth summaries (list of strings) for comparison
    """
    generated_with_newlines = ["\n".join(sent_tokenize(s.strip())) for s in generated]
    reference_with_newlines = ["\n".join(sent_tokenize(s.strip())) for s in reference]
    return rouge_score.compute(
        predictions=generated_with_newlines,
        references=reference_with_newlines,
        use_stemmer=True,
    )

In [0]:
# ROUGE scores for our batch of articles
compute_rouge_score(t5_small_summaries, reference_summaries)

{'rouge1': 0.3100978427358708,
 'rouge2': 0.10558185221314301,
 'rougeL': 0.2217221921741809,
 'rougeLsum': 0.2821954789917822}

## Understanding ROUGE scores

In [0]:
# Sanity check: What if our predictions match the references exactly?
compute_rouge_score(reference_summaries, reference_summaries)

{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}

In [0]:
# And what if we fail to predict anything?
compute_rouge_score(
    generated=["" for _ in range(len(reference_summaries))],
    reference=reference_summaries,
)

{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}

Stemming predictions and references can help to ignore minor differences.

We will use `rouge_score.compute()` directly for these hand-constructed examples.

In [0]:
rouge_score.compute(
    predictions=["Large language models beat world record"],
    references=["Large language models beating world records"],
    use_stemmer=False,
)

{'rouge1': 0.6666666666666666,
 'rouge2': 0.4000000000000001,
 'rougeL': 0.6666666666666666,
 'rougeLsum': 0.6666666666666666}

In [0]:
rouge_score.compute(
    predictions=["Large language models beat world record"],
    references=["Large language models beating world records"],
    use_stemmer=True,
)

{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}

Let's look at how the ROUGE score behaves in various situations.

In [0]:
# What if we predict exactly 1 word correctly?
rouge_score.compute(
    predictions=["Large language models beat world record"],
    references=["Large"],
    use_stemmer=True,
)

{'rouge1': 0.2857142857142857,
 'rouge2': 0.0,
 'rougeL': 0.2857142857142857,
 'rougeLsum': 0.2857142857142857}

In [0]:
# The ROUGE score is symmetric with respect to predictions and references.
rouge_score.compute(
    predictions=["Large"],
    references=["Large language models beat world record"],
    use_stemmer=True,
)

{'rouge1': 0.2857142857142857,
 'rouge2': 0.0,
 'rougeL': 0.2857142857142857,
 'rougeLsum': 0.2857142857142857}

In [0]:
# What about 2 words?  Note how 'rouge1' and 'rouge2' compare with the case when we predict exactly 1 word correctly.
rouge_score.compute(
    predictions=["Large language"],
    references=["Large language models beat world record"],
    use_stemmer=True,
)

{'rouge1': 0.5, 'rouge2': 0.33333333333333337, 'rougeL': 0.5, 'rougeLsum': 0.5}

In [0]:
# Note how rouge1 differs from the rougeN (N>1) scores when we predict word subsequences correctly.
rouge_score.compute(
    predictions=["Models beat large language world record"],
    references=["Large language models beat world record"],
    use_stemmer=True,
)

{'rouge1': 1.0,
 'rouge2': 0.6,
 'rougeL': 0.6666666666666666,
 'rougeLsum': 0.6666666666666666}

d ## Compare small and large models

 We've been working with the `t5-small` model so far.  Let's compare several models with different architectures in terms of their ROUGE scores and some example generated summaries.

In [0]:
def compute_rouge_per_row(
    generated_summaries: list, reference_summaries: list
) -> pd.DataFrame:
    """
    Generates a dataframe to compare rogue score metrics.
    """
    generated_with_newlines = [
        "\n".join(sent_tokenize(s.strip())) for s in generated_summaries
    ]
    reference_with_newlines = [
        "\n".join(sent_tokenize(s.strip())) for s in reference_summaries
    ]
    scores = rouge_score.compute(
        predictions=generated_with_newlines,
        references=reference_with_newlines,
        use_stemmer=True,
        use_aggregator=False,
    )
    scores["generated"] = generated_summaries
    scores["reference"] = reference_summaries
    return pd.DataFrame.from_dict(scores)

### T5-small

The [T5](https://huggingface.co/docs/transformers/model_doc/t5) [[paper]](https://arxiv.org/pdf/1910.10683.pdf) family of models are text-to-text transformers that have been trained on a multi-task mixture of unsupervised and supervised tasks. They are well suited for task such as summarization, translation, text classification, question answering, and more.

The t5-small version of the T5 models has 60 million parameters.

In [0]:
# We computed t5_small_summaries above already.
compute_rouge_score(t5_small_summaries, reference_summaries)

{'rouge1': 0.3100978427358708,
 'rouge2': 0.10558185221314301,
 'rougeL': 0.2217221921741809,
 'rougeLsum': 0.2821954789917822}

In [0]:
t5_small_results = compute_rouge_per_row(
    generated_summaries=t5_small_summaries, reference_summaries=reference_summaries
)
display(t5_small_results)

rouge1,rouge2,rougeL,rougeLsum,generated,reference
0.4074074074074074,0.23076923076923075,0.2962962962962963,0.4074074074074074,"a magnitude 6.7 earthquake rattles Papua new Guinea early Friday afternoon. no tsunami warning was issued, according to the Tsunami Warning Center. papu",Papua New Guinea is on the so-called Ring of Fire . It's on an arc of fault lines that is prone to frequent earthquakes . No tsunami warning was issued .
0.23913043478260868,0.0,0.1956521739130435,0.21739130434782608,the two-Test cricket series is being played in England due to security issues. the series is being played in England due to security issues in Pakistan. the series is being played in,"Australia collapse to 88 all out on opening day of second Test against Pakistan in Leeds . Pakistan, seeking to level two-match series, reached 148-3 when bad light stopped play . Australia captain Ricky Ponting surprisingly chose to bat first in overcast conditions . His team failed to reach three figures in a test for the first time since 1984 ."
0.4545454545454546,0.15625,0.3939393939393939,0.4545454545454546,federal prosecutors want jared Lee Loughner to submit a handwriting sample. prosecutors want the sample to compare with handwritten notes found in his residence. the federal grand,Jared Loughner is refusing the government's request for a writing sample . Authorities want it to compare with notes found in his home after the shooting . Loughner faces 49 charges related to a mass shooting outside a Tucson market .
0.3733333333333333,0.19178082191780824,0.26666666666666666,0.34666666666666673,"new: ""he tried to kill people,"" a 17-year-old student says. new: ""you could feel in the room all the anger that everyone had for him,"" she","Shooting victim McKayla Hicks went to hearing for accused killer James Holmes . She said she could feel ""all the anger that everyone had for"" Holmes . The incident has changed her, said Hicks . A bullet lodged in Hicks' jaw -- doctors said it is safer to leave it there ."
0.2631578947368421,0.10810810810810811,0.1842105263157895,0.21052631578947367,double-amputee sprinter Oscar Pistorius will compete at the 2012 able bodied Olympics. the four-time paralympic gold medalist won a silver,Oscar Pistorius to become first double-amputee Olympian in London . The 25-year-old has been selected in the individual 400 and 4x400m relay . Pistorius had both of his legs amputated when he was 11 months old . He won a silver medal at last year's World Championships in the 4 x 400m relay .
0.2816901408450704,0.057971014492753624,0.19718309859154928,0.2816901408450704,new: a grand jury indicted the governor on charges of coercion of a public servant and abuse of his capacity. new: a grand jury indicted the,"NEW: Perry lawyer calls indictments ""political abuse of the court system"" Indictment by country grand jury in Texas stems from effort to remove local prosecutor . Perry allegedly threatened to veto funding for a program run by the DA in Austin . Indictment could have political implications ."
0.42622950819672134,0.10169491525423728,0.29508196721311475,0.360655737704918,prosecutor says there is enough evidence to continue an investigation. prosecutor Alberto Nisman alleged that he covered up the 1994 bombing. he was found dead in,"Prosecutor to judge: Enough evidence for investigation of President to continue . Different prosecutor, before he died, alleged President hid Iran's alleged involvement in bombing . President Cristina Fernandez de Kirchner, other officials deny cover-up ."
0.20779220779220778,0.08,0.12987012987012986,0.20779220779220778,"UEFA says it is acting over ""the setting-off and throwing of fireworks"" UEFA president calls fans to ""conduct themselves with dignity and respect"" UEFA has already",NEW: UEFA president Michel Platini urges fans to behave at decisive matches Saturday . UEFA says there were racist chants from Croatian fans during a match against Italy . The issue of racism threatens to mar the Euro 2012 s

### T5-base

The [T5-base](https://huggingface.co/t5-base) model has 220 million parameters.

In [0]:
t5_base_summaries = summarize_with_t5(
    model_checkpoint="t5-base", articles=sample["article"]
)
compute_rouge_score(t5_base_summaries, reference_summaries)

{'rouge1': 0.3261538791389793,
 'rouge2': 0.1322216056511341,
 'rougeL': 0.23683757690668544,
 'rougeLsum': 0.30404287772280536}

In [0]:
t5_base_results = compute_rouge_per_row(
    generated_summaries=t5_base_summaries, reference_summaries=reference_summaries
)
display(t5_base_results)

rouge1,rouge2,rougeL,rougeLsum,generated,reference
0.4528301886792453,0.35294117647058826,0.26415094339622636,0.4528301886792453,the quake was centered about 200 miles north-northeast of Port Moresby. no tsunami warning was issued. Papua New Guinea is on the,Papua New Guinea is on the so-called Ring of Fire . It's on an arc of fault lines that is prone to frequent earthquakes . No tsunami warning was issued .
0.3414634146341463,0.12500000000000003,0.1951219512195122,0.29268292682926833,Pakistan reach 148-3 on opening day of two-Test series against australia. teenagers mohammad aamer and mohammad asif take three wickets each,"Australia collapse to 88 all out on opening day of second Test against Pakistan in Leeds . Pakistan, seeking to level two-match series, reached 148-3 when bad light stopped play . Australia captain Ricky Ponting surprisingly chose to bat first in overcast conditions . His team failed to reach three figures in a test for the first time since 1984 ."
0.34782608695652173,0.17910447761194026,0.2608695652173913,0.34782608695652173,federal prosecutors want a handwriting sample from a man accused of killing six people. they want it to compare with notes that include references to guns and bullets. the government,Jared Loughner is refusing the government's request for a writing sample . Authorities want it to compare with notes found in his home after the shooting . Loughner faces 49 charges related to a mass shooting outside a Tucson market .
0.08333333333333333,0.0,0.08333333333333333,0.08333333333333333,"""he's just this freak lookin' dude with some orange hair,"" teen says. ""i think he needs to be killed,"" she says. she says she","Shooting victim McKayla Hicks went to hearing for accused killer James Holmes . She said she could feel ""all the anger that everyone had for"" Holmes . The incident has changed her, said Hicks . A bullet lodged in Hicks' jaw -- doctors said it is safer to leave it there ."
0.37837837837837834,0.16666666666666666,0.21621621621621623,0.32432432432432434,double-amputee sprinter Oscar Pistorius will compete at the 2012 olympics. he was named in the individual 400m and 4x400m,Oscar Pistorius to become first double-amputee Olympian in London . The 25-year-old has been selected in the individual 400 and 4x400m relay . Pistorius had both of his legs amputated when he was 11 months old . He won a silver medal at last year's World Championships in the 4 x 400m relay .
0.3478260869565218,0.1791044776119403,0.31884057971014496,0.3478260869565218,"new: governor's attorney calls indictment a ""political abuse of the court system"" new: he can continue to serve as governor while under indictment.","NEW: Perry lawyer calls indictments ""political abuse of the court system"" Indictment by country grand jury in Texas stems from effort to remove local prosecutor . Perry allegedly threatened to veto funding for a program run by the DA in Austin . Indictment could have political implications ."
0.42622950819672134,0.10169491525423728,0.32786885245901637,0.3934426229508197,a prosecutor says there is enough evidence to continue an investigation into a 1994 bombing. prosecutor Alberto Nisman alleged last month that Fernandez covered up Iran,"Prosecutor to judge: Enough evidence for investigation of President to continue . Different prosecutor, before he died, alleged President hid Iran's alleged involvement in bombing . President Cristina Fernandez de Kirchner, other officials deny cover-up ."
0.23076923076923075,0.02631578947368421,0.12820512820512822,0.23076923076923075,"UEFA opens disciplinary proceedings against Croatia over racist behavior. it says it was acting over ""the setting-off and throwing of fireworks"" and ""the improper conduct of supporters""",NEW: UEFA president Michel Platini urges fans to behave at decisive matches Saturday . UEFA says there were racist chants from Croatian fans during a match against Italy . The issue of racism threatens to mar the Euro 2012 soccer tournament . A dis

### GPT-2

The [GPT-2](https://huggingface.co/gpt2) model is a generative text model that was trained in a self-supervised fashion. Its strengths are in using a 'completing the sentence' for a given prompt.  It has 124 million parameters.

In [0]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer


def summarize_with_gpt2(
    model_checkpoint: str, articles: list, batch_size: int = 8
) -> list:
    """
    Convenience function for summarization with GPT2 to handle these complications:
    - Append "TL;DR" to the end of the input to get GPT2 to generate a summary.
    https://huggingface.co/course/chapter7/5?fw=pt
    - Truncate input to handle long articles.
    - GPT2 uses a max token length of 1024.  We use a shorter 512 limit here.

    :param model_checkpoint: reference to checkpointed model
    :param articles: list of strings
    :return: generated summaries, with the input and "TL;DR" removed
    """
    if torch.cuda.is_available():
        device = "cuda:0"
    else:
        device = "cpu"

    tokenizer = GPT2Tokenizer.from_pretrained(
        model_checkpoint, padding_side="left", cache_dir="/dbfs/mnt/dbacademy-datasets/large-language-models/v01"
    )
    tokenizer.add_special_tokens({"pad_token": tokenizer.eos_token})
    model = GPT2LMHeadModel.from_pretrained(
        model_checkpoint,
        pad_token_id=tokenizer.eos_token_id,
        cache_dir="/dbfs/mnt/dbacademy-datasets/large-language-models/v01",
    ).to(device)

    def perform_inference(batch: list) -> list:
        tmp_inputs = tokenizer(
            batch, max_length=500, return_tensors="pt", padding=True, truncation=True
        )
        tmp_inputs_decoded = tokenizer.batch_decode(
            tmp_inputs.input_ids, skip_special_tokens=True
        )
        inputs = tokenizer(
            [article + " TL;DR:" for article in tmp_inputs_decoded],
            max_length=512,
            return_tensors="pt",
            padding=True,
            truncation=True,
        )
        summary_ids = model.generate(
            inputs.input_ids.to(device),
            attention_mask=inputs.attention_mask.to(device),
            num_beams=2,
            min_length=0,
            max_length=512 + 32,
        )
        return tokenizer.batch_decode(summary_ids, skip_special_tokens=True)

    decoded_summaries = []
    for batch in batch_generator(articles, batch_size=batch_size):
        decoded_summaries += perform_inference(batch)

        # batch clean up
        torch.cuda.empty_cache()
        gc.collect()

    # post-process decoded summaries
    summaries = [
        summary[summary.find("TL;DR:") + len("TL;DR: ") :]
        for summary in decoded_summaries
    ]

    # cleanup
    del tokenizer
    del model
    torch.cuda.empty_cache()
    gc.collect()

    return summaries

In [0]:
gpt2_summaries = summarize_with_gpt2(
    model_checkpoint="gpt2", articles=sample["article"]
)
compute_rouge_score(gpt2_summaries, reference_summaries)

{'rouge1': 0.18885676938991774,
 'rouge2': 0.04373494204085254,
 'rougeL': 0.14094241816970854,
 'rougeLsum': 0.17351838318433643}

In [0]:
gpt2_results = compute_rouge_per_row(
    generated_summaries=gpt2_summaries, reference_summaries=reference_summaries
)
display(gpt2_results)

rouge1,rouge2,rougeL,rougeLsum,generated,reference
0.29508196721311475,0.06779661016949153,0.19672131147540983,0.22950819672131148,"A magnitude 6.7 earthquake rattled Papua New Guinea early Friday afternoon, according to the U.S. Geological Survey. The quake was centered about 200 miles north-northeast of Port Mores",Papua New Guinea is on the so-called Ring of Fire . It's on an arc of fault lines that is prone to frequent earthquakes . No tsunami warning was issued .
0.10869565217391304,0.022222222222222223,0.10869565217391304,0.08695652173913045,"Australia lost the toss. ""I think it was a big mistake,"" Gul said. ""I think it was a big mistake for the team. I think it was a big mistake for the team.","Australia collapse to 88 all out on opening day of second Test against Pakistan in Leeds . Pakistan, seeking to level two-match series, reached 148-3 when bad light stopped play . Australia captain Ricky Ponting surprisingly chose to bat first in overcast conditions . His team failed to reach three figures in a test for the first time since 1984 ."
0.5135135135135135,0.13888888888888887,0.3243243243243243,0.4324324324324324,The government wants to force Jared Lee Loughner to write out something so authorities can view his writing style. The government wants the sample to compare with handwritten notes found in Loughner's residence that,Jared Loughner is refusing the government's request for a writing sample . Authorities want it to compare with notes found in his home after the shooting . Loughner faces 49 charges related to a mass shooting outside a Tucson market .
0.14457831325301204,0.024691358024691357,0.12048192771084339,0.12048192771084339,"I'm not going to be able to do anything about it,"" she said. ""I'm not going to be able to do anything about it. I'm not going to be able to do anything","Shooting victim McKayla Hicks went to hearing for accused killer James Holmes . She said she could feel ""all the anger that everyone had for"" Holmes . The incident has changed her, said Hicks . A bullet lodged in Hicks' jaw -- doctors said it is safer to leave it there ."
0.24444444444444444,0.04545454545454545,0.2,0.24444444444444444,"""The CAS has determined that the athlete's performance in the event of a disqualification is not in the public interest, and that the athlete's performance in the event of a disqualification is not in",Oscar Pistorius to become first double-amputee Olympian in London . The 25-year-old has been selected in the individual 400 and 4x400m relay . Pistorius had both of his legs amputated when he was 11 months old . He won a silver medal at last year's World Championships in the 4 x 400m relay .
0.1,0.0,0.1,0.1,"I'm not going to resign,"" Lehmberg said in a statement to CNN. ""I'm not going to resign because I'm not going to be a good district attorney. I'm not going to","NEW: Perry lawyer calls indictments ""political abuse of the court system"" Indictment by country grand jury in Texas stems from effort to remove local prosecutor . Perry allegedly threatened to veto funding for a program run by the DA in Austin . Indictment could have political implications ."
0.09302325581395347,0.0,0.04651162790697674,0.09302325581395347,Fernandez's death was a suicide. Read More,"Prosecutor to judge: Enough evidence for investigation of President to continue . Different prosecutor, before he died, alleged President hid Iran's alleged involvement in bombing . President Cristina Fernandez de Kirchner, other officials deny cover-up ."
0.28571428571428564,0.12195121951219513,0.19047619047619052,0.28571428571428564,"""I'm not a racist, I'm a good player."" The Czech Republic has also been fined $1,000 for racist chanting during a match against Italy on Saturday. The Czech Republic has also",NEW: UEFA president Michel Platini urges fans to behave at decisive matches Saturday . UEFA says there were racist chants from Croatian fans during a match against Italy . The issue of racism threatens to mar the Euro 2012 soccer tou

### Comparing all models

We use a couple of helper functions to compare the above models, first by their evaluation metrics (quantitative) and second by their generated summaries (qualitative).

In [0]:
def compare_models(models_results: dict) -> pd.DataFrame:
    """
    :param models_results: dict of "model name" string mapped to pd.DataFrame of results computed by `compute_rouge_per_row`
    :return: pd.DataFrame with 1 row per model, with columns: model, rouge1, rouge2, rougeL, rougeLsum
    where metrics are averages over input results for each model
    """
    agg_results = []
    for r in models_results:
        model_results = models_results[r].drop(
            labels=["generated", "reference"], axis=1
        )
        agg_metrics = [r]
        agg_metrics[1:] = model_results.mean(axis=0)
        agg_results.append(agg_metrics)
    return pd.DataFrame(
        agg_results, columns=["model", "rouge1", "rouge2", "rougeL", "rougeLsum"]
    )

In [0]:
display(
    compare_models(
        {
            "t5-small": t5_small_results,
            "t5-base": t5_base_results,
            "gpt2": gpt2_results,
        }
    )
)

model,rouge1,rouge2,rougeL,rougeLsum
t5-small,0.3096417280107126,0.10613253877927877,0.22150868083285702,0.2821843556987063
t5-base,0.32710299231055473,0.13328785373365565,0.23726430250115693,0.303475912290666
gpt2,0.18859605456462322,0.043728564598415474,0.14158336222792456,0.1728018445721218


In [0]:
def compare_models_summaries(models_summaries: dict) -> pd.DataFrame:
    """
    Aggregates results from `models_summaries` and returns a dataframe.
    """
    comparison_df = None
    for model_name in models_summaries:
        summaries_df = models_summaries[model_name]
        if comparison_df is None:
            comparison_df = summaries_df[["generated"]].rename(
                {"generated": model_name}, axis=1
            )
        else:
            comparison_df = comparison_df.join(
                summaries_df[["generated"]].rename({"generated": model_name}, axis=1)
            )
    return comparison_df

In [0]:
# In the output table below, scroll to the right to see all models.
display(
    compare_models_summaries(
        {
            "t5_small": t5_small_results,
            "t5_base": t5_base_results,
            "gpt2": gpt2_results,
        }
    )
)

t5_small,t5_base,gpt2
"a magnitude 6.7 earthquake rattles Papua new Guinea early Friday afternoon. no tsunami warning was issued, according to the Tsunami Warning Center. papu",the quake was centered about 200 miles north-northeast of Port Moresby. no tsunami warning was issued. Papua New Guinea is on the,"A magnitude 6.7 earthquake rattled Papua New Guinea early Friday afternoon, according to the U.S. Geological Survey. The quake was centered about 200 miles north-northeast of Port Mores"
the two-Test cricket series is being played in England due to security issues. the series is being played in England due to security issues in Pakistan. the series is being played in,Pakistan reach 148-3 on opening day of two-Test series against australia. teenagers mohammad aamer and mohammad asif take three wickets each,"Australia lost the toss. ""I think it was a big mistake,"" Gul said. ""I think it was a big mistake for the team. I think it was a big mistake for the team."
federal prosecutors want jared Lee Loughner to submit a handwriting sample. prosecutors want the sample to compare with handwritten notes found in his residence. the federal grand,federal prosecutors want a handwriting sample from a man accused of killing six people. they want it to compare with notes that include references to guns and bullets. the government,The government wants to force Jared Lee Loughner to write out something so authorities can view his writing style. The government wants the sample to compare with handwritten notes found in Loughner's residence that
"new: ""he tried to kill people,"" a 17-year-old student says. new: ""you could feel in the room all the anger that everyone had for him,"" she","""he's just this freak lookin' dude with some orange hair,"" teen says. ""i think he needs to be killed,"" she says. she says she","I'm not going to be able to do anything about it,"" she said. ""I'm not going to be able to do anything about it. I'm not going to be able to do anything"
double-amputee sprinter Oscar Pistorius will compete at the 2012 able bodied Olympics. the four-time paralympic gold medalist won a silver,double-amputee sprinter Oscar Pistorius will compete at the 2012 olympics. he was named in the individual 400m and 4x400m,"""The CAS has determined that the athlete's performance in the event of a disqualification is not in the public interest, and that the athlete's performance in the event of a disqualification is not in"
new: a grand jury indicted the governor on charges of coercion of a public servant and abuse of his capacity. new: a grand jury indicted the,"new: governor's attorney calls indictment a ""political abuse of the court system"" new: he can continue to serve as governor while under indictment.","I'm not going to resign,"" Lehmberg said in a statement to CNN. ""I'm not going to resign because I'm not going to be a good district attorney. I'm not going to"
prosecutor says there is enough evidence to continue an investigation. prosecutor Alberto Nisman alleged that he covered up the 1994 bombing. he was found dead in,a prosecutor says there is enough evidence to continue an investigation into a 1994 bombing. prosecutor Alberto Nisman alleged last month that Fernandez covered up Iran,Fernandez's death was a suicide. Read More
"UEFA says it is acting over ""the setting-off and throwing of fireworks"" UEFA president calls fans to ""conduct themselves with dignity and respect"" UEFA has already","UEFA opens disciplinary proceedings against Croatia over racist behavior. it says it was acting over ""the setting-off and throwing of fireworks"" and ""the improper conduct of supporters""","""I'm not a racist, I'm a good player."" The Czech Republic has also been fined $1,000 for racist chanting during a match against Italy on Saturday. The Czech Republic has also"
new: a new high-level group will discuss economic and trade relations between the two nations. new: the presidents will meet with the president on friday. new: the,"""that's

-sandbox
&copy; 2023 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>